In [1]:
import polars as pl

In [3]:
survey = pl.read_csv("data/health_survey.csv")

In [16]:
recode = pl.read_csv("data/reverse_coding_items.csv")

In [11]:
item_cols = [c for c in survey.columns if c != "id"]
item_cols

['F1',
 'F5',
 'F2',
 'F1.1',
 'F2.1',
 'F6',
 'F4',
 'F3',
 'F5.1',
 'F1.2',
 'F2.2',
 'F6.1',
 'F2.3',
 'F4.1',
 'F2.4',
 'F5.2',
 'F2.5',
 'F6.2',
 'F1.3',
 'F2.6',
 'F5.3',
 'F4.2',
 'F2.7',
 'F3.1',
 'F2.8',
 'F5.4',
 'F3.2',
 'F1.4',
 'F3.3',
 'F1.5',
 'F5.5',
 'F6.3',
 'F1.6',
 'F5.6',
 'F2.9',
 'F3.4',
 'F4.3',
 'F2.10',
 'F1.7',
 'F6.4',
 'F4.4',
 'F5.7',
 'F3.5',
 'F2.11']

In [12]:
long = survey.unpivot(
    index=["id"],
    on=item_cols,
    variable_name="Question",
    value_name="Response"
)
long

id,Question,Response
i64,str,str
1,"""F1""","""Somewhat Agree"""
2,"""F1""","""Somewhat Agree"""
3,"""F1""","""Strongly Agree"""
4,"""F1""","""Somewhat Agree"""
5,"""F1""","""Strongly Agree"""
…,…,…
260,"""F2.11""","""Neither Agree nor Disagree"""
261,"""F2.11""","""Somewhat Agree"""
262,"""F2.11""","""Somewhat Agree"""


In [17]:
recode.columns

['Question',
 'Construct',
 'Question # on Qualtrics Survey',
 'Needs Reverse Coding?',
 'Column Name']

In [19]:
recode_select = (
    recode.select([
        pl.col("Column Name").alias("Question"),
        pl.col("Needs Reverse Coding?").alias("Needs Reverse")
    ])
)
recode_select

Question,Needs Reverse
str,str
"""F1""","""No"""
"""F5""","""Yes"""
"""F2""","""No"""
"""F1.1""","""No"""
"""F2.1""","""No"""
…,…
"""F6.4""","""Yes"""
"""F4.4""","""No"""
"""F5.7""","""No"""


In [22]:
recode_long = long.join(recode_select, on="Question", how="left").with_columns(
    pl.col("Needs Reverse").fill_null("No")
)
recode_long

id,Question,Response,Needs Reverse
i64,str,str,str
1,"""F1""","""Somewhat Agree""","""No"""
2,"""F1""","""Somewhat Agree""","""No"""
3,"""F1""","""Strongly Agree""","""No"""
4,"""F1""","""Somewhat Agree""","""No"""
5,"""F1""","""Strongly Agree""","""No"""
…,…,…,…
260,"""F2.11""","""Neither Agree nor Disagree""","""No"""
261,"""F2.11""","""Somewhat Agree""","""No"""
262,"""F2.11""","""Somewhat Agree""","""No"""


In [24]:
recode_long = recode_long.with_columns(
    pl.col("Needs Reverse")
        .cast(pl.Utf8)
        .str.strip_chars()
        .str.to_lowercase()
        .is_in(["yes", "y", "true", "1"])
        .alias("needs_reverse_bool")
)
recode_long

id,Question,Response,Needs Reverse,needs_reverse_bool
i64,str,str,str,bool
1,"""F1""","""Somewhat Agree""","""No""",false
2,"""F1""","""Somewhat Agree""","""No""",false
3,"""F1""","""Strongly Agree""","""No""",false
4,"""F1""","""Somewhat Agree""","""No""",false
5,"""F1""","""Strongly Agree""","""No""",false
…,…,…,…,…
260,"""F2.11""","""Neither Agree nor Disagree""","""No""",false
261,"""F2.11""","""Somewhat Agree""","""No""",false
262,"""F2.11""","""Somewhat Agree""","""No""",false


In [25]:
recode_long = recode_long.with_columns(
    pl.when(pl.col("Response") == "Strongly Disagree").then(1)
     .when(pl.col("Response") == "Somewhat Disagree").then(2)
     .when(pl.col("Response") == "Neither Agree nor Disagree").then(3)
     .when(pl.col("Response") == "Somewhat Agree").then(4)
     .when(pl.col("Response") == "Strongly Agree").then(5)
     .otherwise(None)
     .alias("TempRecode Value")
)
recode_long

id,Question,Response,Needs Reverse,needs_reverse_bool,TempRecode Value
i64,str,str,str,bool,i32
1,"""F1""","""Somewhat Agree""","""No""",false,4
2,"""F1""","""Somewhat Agree""","""No""",false,4
3,"""F1""","""Strongly Agree""","""No""",false,5
4,"""F1""","""Somewhat Agree""","""No""",false,4
5,"""F1""","""Strongly Agree""","""No""",false,5
…,…,…,…,…,…
260,"""F2.11""","""Neither Agree nor Disagree""","""No""",false,3
261,"""F2.11""","""Somewhat Agree""","""No""",false,4
262,"""F2.11""","""Somewhat Agree""","""No""",false,4


In [26]:
recode_long = recode_long.with_columns(
    pl.when(pl.col("needs_reverse_bool"))
      .then(6 - pl.col("TempRecode Value"))
      .otherwise(pl.col("TempRecode Value"))
      .alias("TempReverse Value")
)
recode_long

id,Question,Response,Needs Reverse,needs_reverse_bool,TempRecode Value,TempReverse Value
i64,str,str,str,bool,i32,i32
1,"""F1""","""Somewhat Agree""","""No""",false,4,4
2,"""F1""","""Somewhat Agree""","""No""",false,4,4
3,"""F1""","""Strongly Agree""","""No""",false,5,5
4,"""F1""","""Somewhat Agree""","""No""",false,4,4
5,"""F1""","""Strongly Agree""","""No""",false,5,5
…,…,…,…,…,…,…
260,"""F2.11""","""Neither Agree nor Disagree""","""No""",false,3,3
261,"""F2.11""","""Somewhat Agree""","""No""",false,4,4
262,"""F2.11""","""Somewhat Agree""","""No""",false,4,4


In [27]:
recode_long = recode_long.with_columns(
      pl.col("Question").cast(pl.Utf8).str.split(".").list.first().alias("Question Type")
)
recode_long

id,Question,Response,Needs Reverse,needs_reverse_bool,TempRecode Value,TempReverse Value,Question Type
i64,str,str,str,bool,i32,i32,str
1,"""F1""","""Somewhat Agree""","""No""",false,4,4,"""F1"""
2,"""F1""","""Somewhat Agree""","""No""",false,4,4,"""F1"""
3,"""F1""","""Strongly Agree""","""No""",false,5,5,"""F1"""
4,"""F1""","""Somewhat Agree""","""No""",false,4,4,"""F1"""
5,"""F1""","""Strongly Agree""","""No""",false,5,5,"""F1"""
…,…,…,…,…,…,…,…
260,"""F2.11""","""Neither Agree nor Disagree""","""No""",false,3,3,"""F2"""
261,"""F2.11""","""Somewhat Agree""","""No""",false,4,4,"""F2"""
262,"""F2.11""","""Somewhat Agree""","""No""",false,4,4,"""F2"""


In [30]:
aggregation = (
    recode_long
    .group_by(["id", "Question Type"])
    .agg(pl.col("TempReverse Value").sum().alias("Recoded Total"))
)
aggregation

id,Question Type,Recoded Total
i64,str,i32
120,"""F5""",32
183,"""F2""",48
249,"""F4""",16
130,"""F1""",35
135,"""F5""",32
…,…,…
145,"""F5""",29
59,"""F4""",16
30,"""F1""",31


In [35]:
summmary = (
    aggregation
    .pivot(
        values="Recoded Total",
        index="id",
        on="Question Type",
        aggregate_function="first"
    )
    .sort("id")
)
summmary

id,F5,F2,F4,F1,F6,F3
i64,i32,i32,i32,i32,i32,i32
1,28,48,17,31,18,20
2,27,47,17,31,20,19
3,32,46,18,36,17,19
4,30,54,15,32,16,12
5,36,47,19,37,19,22
…,…,…,…,…,…,…
260,34,50,18,32,22,22
261,28,48,19,31,15,24
262,33,47,23,30,19,20


In [36]:
summmary.write_csv("health_survey_summary.csv")

# Why use relative addresses?

In this notebook, we will illustrate 

1. That relative addresses for loading data files works, but
2. Using absolute addresses for loading data files *will not*.

## Problem 1 - Load the `lat_long_example.csv` file using a relative address.

**Tasks.**
1. Open a terminal, start `nu`, and navigate to the root menu of your first/primary data repository,
2. Use `ls **/*` to get the relative address of `lat_long_example.csv`, and
3. Use `polars` to load and inspect these data using this relative path.

In [10]:
relative_path = " ... Your path here ..."

In [ ]:
(lat_lng_example :=
 pl.read_csv(...)
)

## Problem 2 - Load the `lat_long_example.csv` file using a absolute address.

**Tasks.**
1. Open a terminal, start `nu`, and navigate to the root menu of *one of your first/primary data repository,
2. Use `glob **/*` to get the absolute address of `lat_long_example.csv`, and
3. Use `polars` to load and inspect these data.

In [5]:
absolute_path = " ... Your path here ..."

In [ ]:
(lat_lng_example :=
 pl.read_csv(...)
)

## Illustrating the problem with absolute addresses

While the relative address in problem 1 points to the data IN THIS COPY of the repo, the absolute address points to the data in EXACTLY one of the copies of the repository. This becomes a problem if (A) anything changes in that repository, or (B) we are working on a different machine.

**Tasks.** To illustrate why this is a problem, do the following.

1. From your first/primary repository commit and push this notebook to GitHub,
2. Fetch and pull this notebook to another local copy of the repository,
3. In your file explorer (Files or Finder), move your first/primary repository into another folder, e.g., make a new folder and drag-and-drop the repo.
4. Rerun the cells in each local copy of the repository and document your findings in the WORD document. 